# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from datetime import datetime

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, pipeline
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import AutoModelForCausalLM, AutoModel
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.llms import OpenAI

# Initial Architecture

## Getting from csv and splitting

In [21]:
csv_file_path = 'example_data.csv'

loader = CSVLoader(file_path=csv_file_path)

data = loader.load()

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

docs = text_splitter.split_documents(data)

In [23]:
docs[0]

Document(metadata={'source': 'example_data.csv', 'row': 0}, page_content=': \ncategory: Eğitim 101\nmain_title: Ders Seçimi\nsub_title: main text\ncontent: ODTÜ 2022-2023 yılı sonbahar dönemi için etkileşimli kayıt sistemi 28 Eylül 2022 tarihinde, saat 09.00’da öğrencilere açılacak. Birer saat arayla, başta 4. sınıflar olmak üzere tüm sınıflara kademeli olarak açılmaya devam edecek. Etkileşimli kayıtlar 30 Eylül, saat 17.00’de sona erecektir. Danışman onayları ise 28 Eylül’de başlayıp 30 Eylül saat 23.59’da sona erecektir. Not: Kayıt programına kampüs dışından erişimlerde VPN bağlantısı şartı bulunmamaktadır.')

## Getting an pre-trained embedding model

In [24]:
model_path = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name = model_path,      
    model_kwargs = model_kwargs,  
    encode_kwargs = encode_kwargs 
)

In [25]:
text = 'Üreten ekibe hoş geldin'
query_result = embeddings.embed_query(text)
len(query_result)

384

## Creating a vector database

In [26]:
vector_db = FAISS.from_documents(docs, embeddings)

In [27]:
question = "metubot kim tarafından geliştirilmektedir"
searchDocs = vector_db.similarity_search(question)
print(searchDocs[0])

page_content=': 
category: Deneme
main_title: Topluluklar
sub_title: Topluluklar
content: metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.' metadata={'source': 'example_data.csv', 'row': 102}


## Creating a retriever

In [28]:
retriever = vector_db.as_retriever()

docs = retriever.invoke("metubot kim tarafından geliştirilmektedir")
print(docs[0])

page_content=': 
category: Deneme
main_title: Topluluklar
sub_title: Topluluklar
content: metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.' metadata={'source': 'example_data.csv', 'row': 102}


# Llama 3.2 3b

In [29]:
llama_llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.1,
)

In [30]:
llama_qa = RetrievalQA.from_chain_type(
    llm=llama_llm,
    chain_type="stuff",
    retriever=retriever
)

In [31]:
docs = retriever.invoke(question)
print(docs[0])

page_content=': 
category: Deneme
main_title: Topluluklar
sub_title: Topluluklar
content: metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.' metadata={'source': 'example_data.csv', 'row': 102}


In [32]:
start_time = datetime.now()

result = llama_qa.invoke(question)
print(result['result'])

end_time = datetime.now()

Metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.


In [33]:
elapsed_time = end_time - start_time
print(f"Geçen Süre: {elapsed_time}")

Geçen Süre: 0:00:00.795603


# Opus en-tr

In [34]:
en_tr_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-tr", device="cuda")

# Defining the answering function

In [35]:
question = "Answer in English based on the information given, act as a personal assistant. Metubot'u kim geliştiriyor"

In [36]:
start_time = datetime.now()

result = llama_qa.invoke(question)

mid_time = datetime.now()

result_tr = en_tr_pipe(result["result"])

end_time = datetime.now()

print(result_tr[0]["translation_text"])

MetuBot'u kimin geliştirdiğinden emin değilim. Doğru bir cevap vermek için yeterli bilgiye sahip değilim. Daha fazla bağlam sağlayabilir veya ne aradığınızı açıklığa kavuşturabilirseniz, daha fazla denemekten ve yardımcı olmaktan mutluluk duyarım!


In [37]:
llama_time = mid_time - start_time
translation_time = end_time - mid_time
top_time = end_time - start_time
print(f"Llama answer time: {llama_time}\nOpus answer_time: {translation_time}\nFull time: {top_time}")

Llama answer time: 0:00:01.197517
Opus answer_time: 0:00:00.201186
Full time: 0:00:01.398703


# Conclusion

Llama, when coupled with a translator, produced grammatically accurate and contextually appropriate text. However, Llama struggled to fully understand the contextual implications of the information retrieved.